In [2]:
import sqlite3
import pandas as pd

In [3]:
PATH = "C:\\Users\\visha\\Downloads\\Subtitle_Seachengine\\Subtitle_Search_Engine-main\\Subtitle_Search_Engine-main\\eng_subtitles_database.db\\eng_subtitles_database.db"

In [4]:
conn = sqlite3.connect(PATH)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('zipfiles',)]


In [5]:
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

num
name
content


In [6]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df.head()

,num,name,content
0,9180533,the.message.(1976).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x1c\xa9\x...
1,9180583,here.comes.the.grump.s01.e09.joltin.jack.in.bo...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x17\xb9\x...
2,9180592,yumis.cells.s02.e13.episode.2.13.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00L\xb9\x99V...
3,9180594,yumis.cells.s02.e14.episode.2.14.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x00U\xa9\x99V...
4,9180600,broker.(2022).eng.1cd,b'PK\x03\x04\x14\x00\x00\x00\x08\x001\xa9\x99V...


In [7]:
df.shape

(82498, 3)

In [8]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 82498 entries, 0 to 82497
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      82498 non-null  int64 
 1   name     82498 non-null  object
 2   content  82498 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.8 GB


In [9]:
if df['num'].dtype == 'int64' and df['num'].max() < 2**31:
    # Reduce num to int32 if safe (avoid overflow)
    df['num'] = df['num'].astype('int32')

In [10]:
selected_df = df.sample(frac=0.30, random_state=42)
selected_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 24749 entries, 17262 to 67859
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      24749 non-null  int32 
 1   name     24749 non-null  object
 2   content  24749 non-null  object
dtypes: int32(1), object(2)
memory usage: 550.9 MB


In [11]:
import zipfile
import io

count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])

    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text

In [12]:
selected_df['file_content'] = selected_df['content'].apply(decode_method)

In [13]:
selected_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 24749 entries, 17262 to 67859
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   num           24749 non-null  int32 
 1   name          24749 non-null  object
 2   content       24749 non-null  object
 3   file_content  24749 non-null  object
dtypes: int32(1), object(3)
memory usage: 1.9 GB


In [14]:
selected_df = selected_df.drop('content', axis=1)
selected_df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Index: 24749 entries, 17262 to 67859
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   num           24749 non-null  int32 
 1   name          24749 non-null  object
 2   file_content  24749 non-null  object
dtypes: int32(1), object(2)
memory usage: 1.4 GB


In [15]:
selected_df.head(2)

,num,name,file_content
17262,9251120,maybe.this.time.(2014).eng.1cd,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch..."
7294,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,"1\r\n00:00:09,275 --> 00:00:11,876\r\n¶ Oh, I ..."


# Now extract the content

In [16]:
import re

def clean_text(text):
    text = re.sub(r'\d{2}:\d{2}:\d{2},\d{3} --> \d{2}:\d{2}:\d{2},\d{3}\r\n', '', text)
    text = re.sub(r'\r\n', ' ', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text)
    text = text.strip()
    return text

In [17]:
selected_df['file_content'] = selected_df['file_content'].apply(clean_text)

In [18]:
# selected_df.head()
selected_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 24749 entries, 17262 to 67859
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   num           24749 non-null  int32 
 1   name          24749 non-null  object
 2   file_content  24749 non-null  object
dtypes: int32(1), object(2)
memory usage: 612.4 MB


# Let's optimise the dataset more

In [19]:
# Assuming your DataFrame is named 'selected_df'
new_df = selected_df.copy()  # Create a copy to avoid modifying the original

# Rename columns
new_df = new_df.rename(columns={'num': 'subtitle_id', 'name': 'subtitle_name', 'file_content': 'content'})

# Convert 'subtitle_id' to string type (assuming it's currently numerical)
new_df['subtitle_id'] = new_df['subtitle_id'].astype(str)

In [20]:
new_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 24749 entries, 17262 to 67859
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   subtitle_id    24749 non-null  object
 1   subtitle_name  24749 non-null  object
 2   content        24749 non-null  object
dtypes: object(3)
memory usage: 613.8 MB


In [21]:
new_df.head()

,subtitle_id,subtitle_name,content
17262,9251120,maybe.this.time.(2014).eng.1cd,Watch any video online with OpenSUBTITLES Free...
7294,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,Oh I know that its getting late but I dont wan...
47707,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,Timing and Subtitles by The Uncontrollable Lov...
29914,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,ethereal music apiOpenSubtitlesorg is deprecat...
54266,9408707,battlebots.(2015).eng.1cd,Chris Oh no not the Minibots yelling Oh You le...


In [22]:
new_df = new_df.reset_index(drop=True)
new_df.head()

,subtitle_id,subtitle_name,content
0,9251120,maybe.this.time.(2014).eng.1cd,Watch any video online with OpenSUBTITLES Free...
1,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,Oh I know that its getting late but I dont wan...
2,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,Timing and Subtitles by The Uncontrollable Lov...
3,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,ethereal music apiOpenSubtitlesorg is deprecat...
4,9408707,battlebots.(2015).eng.1cd,Chris Oh no not the Minibots yelling Oh You le...


In [23]:
# Let's check out the dataset
new_df['content'][0]

'Watch any video online with OpenSUBTITLES Free Browser extension osdblinkext It couldve been just another summer But as I set foot on the sand that summer suddenly felt different Like it was going to be the summer that would change my life The summer of freedom The summer of endless possibilities The summer of Ooh aah Ooh oh Oh oh ooh That was the summer of you and me Youre quite the dancer Why did you stop Come on Keep dancing Whatever Im kidding Dont get mad Huh What Hey Im just going to get my towel What Stop that You thought I was gonna kiss you No Excuse me I wanna kiss you but not just yet What do you mean not yet Only when youre my girl What do you mean your girl My girlfriend Miss As if You wish And dont call me miss Dont pretend to be a gentleman when youre clearly not So what should I call you Rude Snob Bitch And you Douche Handsome Conceited Just like you Huh Jerk Exactly your type Leave me alone Steph Aha Steph Ill just call you Tep Remove the S and the F By the way Im Ton

# Let's save the new dataset

In [24]:
!pip install pyarrow -q

In [25]:
path = "C:\\Users\\visha\Downloads\\Subtitle_Seachengine\\Subtitle_Search_Engine-main\\Subtitle_Search_Engine-main\\Data"

# save the file as parquet file for better memory efficieny as we have system constraints.
new_df.to_parquet(f"{path}subtitles.parquet")

# Now, let's do the data chunking part

In [26]:
def partitioning_srt_file_data_into_chunks(row, window_size=512, overlapping=100):
    chunks = []
    i = 0
    while i < len(row['content']):
        chunk_end = min(i + window_size, len(row['content']))
        chunks.append(row['content'][i:chunk_end])
        i += window_size - overlapping
    return chunks

In [27]:
new_df['content'] = new_df.apply(partitioning_srt_file_data_into_chunks, axis=1)

In [28]:
new_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24749 entries, 0 to 24748
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   subtitle_id    24749 non-null  object
 1   subtitle_name  24749 non-null  object
 2   content        24749 non-null  object
dtypes: object(3)
memory usage: 18.6 MB


In [29]:
new_df.head()

,subtitle_id,subtitle_name,content
0,9251120,maybe.this.time.(2014).eng.1cd,[Watch any video online with OpenSUBTITLES Fre...
1,9211589,down.the.shore.s01.e10.and.justice.for.all.(19...,[Oh I know that its getting late but I dont wa...
2,9380845,uncontrollably.fond.s01.e07.heartache.(2016).e...,[Timing and Subtitles by The Uncontrollable Lo...
3,9301436,screen.two.s13.e04.the.precious.blood.(1996).e...,[ethereal music apiOpenSubtitlesorg is depreca...
4,9408707,battlebots.(2015).eng.1cd,[Chris Oh no not the Minibots yelling Oh You l...


In [30]:
new_df['content'][0]

['Watch any video online with OpenSUBTITLES Free Browser extension osdblinkext It couldve been just another summer But as I set foot on the sand that summer suddenly felt different Like it was going to be the summer that would change my life The summer of freedom The summer of endless possibilities The summer of Ooh aah Ooh oh Oh oh ooh That was the summer of you and me Youre quite the dancer Why did you stop Come on Keep dancing Whatever Im kidding Dont get mad Huh What Hey Im just going to get my towel What',
 'ome on Keep dancing Whatever Im kidding Dont get mad Huh What Hey Im just going to get my towel What Stop that You thought I was gonna kiss you No Excuse me I wanna kiss you but not just yet What do you mean not yet Only when youre my girl What do you mean your girl My girlfriend Miss As if You wish And dont call me miss Dont pretend to be a gentleman when youre clearly not So what should I call you Rude Snob Bitch And you Douche Handsome Conceited Just like you Huh Jerk Exact

In [31]:
# Let's use the concept of explode to create a seperate the content chunks
new_df = new_df.explode('content', ignore_index=False)

In [32]:
new_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 1560370 entries, 0 to 24748
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype 
---  ------         --------------    ----- 
 0   subtitle_id    1560370 non-null  object
 1   subtitle_name  1560370 non-null  object
 2   content        1560370 non-null  object
dtypes: object(3)
memory usage: 1.1 GB


In [33]:
extracted_df = new_df.sample(frac=0.3, random_state=42, axis=0)

In [34]:
extracted_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 468111 entries, 3536 to 371
Data columns (total 3 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   subtitle_id    468111 non-null  object
 1   subtitle_name  468111 non-null  object
 2   content        468111 non-null  object
dtypes: object(3)
memory usage: 331.0 MB


In [36]:
# Now save the chunked dataset
path = "C:\\Users\\visha\Downloads\\Subtitle_Seachengine\\Subtitle_Search_Engine-main\\Subtitle_Search_Engine-main\\Data"
extracted_df.to_parquet(f"{path}subtitles_extracted.parquet")